# Install and Imports

In [1]:
!pip install -q pandasql imbalanced-learn xgboost lightgbm wordcloud nltk emoji


import os
import re
import string
import numpy as np
import pandas as pd
from collections import Counter
from datetime import datetime

# ML imports
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
import xgboost as xgb
import lightgbm as lgb

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from wordcloud import WordCloud
from sklearn.metrics import precision_recall_fscore_support


try:
    from pandasql import sqldf
    pysqldf = lambda q: sqldf(q, globals())
    print("✅ pandasql imported successfully")
except ImportError:
    print("⚠️ pandasql not installed; skipping SQL-style queries")



DATA_PATH = "r_dataisbeautiful_posts.csv"
RANDOM_STATE = 42


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 10.4 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ pandasql imported successfully


# Data Loading

In [2]:
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
display(df.head(3))
display(df.info())


Shape: (193091, 12)


/tmp/ipython-input-2430322758.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DATA_PATH)


,id,title,score,author,author_flair_text,removed_by,total_awards_received,awarders,created_utc,full_link,num_comments,over_18
0,it7yz2,[OC] The World’s Forests Mapped,1,vividmaps,NaN,NaN,0.0,[],1600173678,https://www.reddit.com/r/dataisbeautiful/comme...,1,False
1,it7uig,[OC] Airbnb presence mapped in Barcelona (BCN)...,1,JonnieNeptune,OC: 2,NaN,0.0,[],1600173208,https://www.reddit.com/r/dataisbeautiful/comme...,0,False
2,it7t4z,A marketplace for open streaming data sources,1,DangerMouse289,NaN,automod_filtered,0.0,[],1600173075,https://www.reddit.com/r/dataisbeautiful/comme...,0,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193091 entries, 0 to 193090
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     193091 non-null  object 
 1   title                  193090 non-null  object 
 2   score                  193091 non-null  int64  
 3   author                 193091 non-null  object 
 4   author_flair_text      26873 non-null   object 
 5   removed_by             15685 non-null   object 
 6   total_awards_received  53239 non-null   float64
 7   awarders               42568 non-null   object 
 8   created_utc            193091 non-null  int64  
 9   full_link              193091 non-null  object 
 10  num_comments           193091 non-null  int64  
 11  over_18                193091 non-null  bool   
dtypes: bool(1), float64(1), int64(3), object(7)
memory usage: 16.4+ MB


None

# Exploratory Data Analysis

In [3]:
print("Missing per column:\n", df.isna().sum().sort_values(ascending=False).head(20))
print("\nDtypes:\n", df.dtypes.value_counts())

# Example SQL-style queries with pandasql:
# 1) get counts by subreddit if column exists
if 'subreddit' in df.columns:
    q = """
    SELECT subreddit, COUNT(*) as cnt, AVG(score) as avg_score
    FROM df
    GROUP BY subreddit
    HAVING COUNT(*) > 50
    ORDER BY avg_score DESC
    LIMIT 20
    """
    print("Top subreddits by avg score (count>50):")
    display(pysqldf(q))
else:
    print("No subreddit column found; skipping subreddit SQL sample.")


Missing per column:
 removed_by               177406
author_flair_text        166218
awarders                 150523
total_awards_received    139852
title                         1
author                        0
id                            0
score                         0
created_utc                   0
full_link                     0
num_comments                  0
over_18                       0
dtype: int64

Dtypes:
 object     7
int64      3
float64    1
bool       1
Name: count, dtype: int64
No subreddit column found; skipping subreddit SQL sample.


# Identifying Target Column

In [4]:
target_col = 'removed_by'

# Convert to binary target: 1 = removed, 0 = visible
df[target_col] = df[target_col].notnull().astype(int)

print("Using target column:", target_col)
print(df[target_col].value_counts(dropna=False))
print("Percent positive (removed):", round(df[target_col].mean()*100, 2), "%")


Using target column: removed_by
removed_by
0    177406
1     15685
Name: count, dtype: int64
Percent positive (removed): 8.12 %


# Numeric Correlations with Target

In [5]:
df['_target_bin'] = df['removed_by']

# Select numeric columns
numcols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", numcols)

# Compute Pearson correlations
corr = df[numcols].corr()['removed_by'].sort_values(ascending=False)
print("\nTop correlations with target (removed_by):")
display(corr.head(20))


Numeric columns: ['score', 'removed_by', 'total_awards_received', 'created_utc', 'num_comments', '_target_bin']

Top correlations with target (removed_by):


,removed_by
removed_by,1.000000
_target_bin,1.000000
created_utc,0.391379
total_awards_received,-0.010830
score,-0.026245
num_comments,-0.034130


# Preprocessing and Feature Engineering

In [6]:
import numpy as np
import pandas as pd
import re, string, nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')


# Load data
df = pd.read_csv("r_dataisbeautiful_posts.csv")
print("Data shape:", df.shape)

# Define text and target columns
text_col = 'title'
target_col = 'removed_by'

# Binary target: 1 if removed_by not null, else 0
df[target_col] = df[target_col].notnull().astype(int)

# Fill missing text
df[text_col] = df[text_col].fillna("")

# Numeric features to use
meta_cols = ['score', 'num_comments', 'total_awards_received', 'created_utc']
for c in meta_cols:
    if c in df.columns:
        df[c] = df[c].fillna(df[c].median())

# Basic feature engineering
df['text_len'] = df[text_col].apply(len)
df['word_count'] = df[text_col].apply(lambda x: len(x.split()))

# Clean text
STOP = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(s):
    s = str(s).lower()
    s = re.sub(r'http\S+|www.\S+', ' ', s)
    s = re.sub(r'\[oc\]', ' ', s)
    s = re.sub(r'[^a-z\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def stem_text(s):
    return " ".join([stemmer.stem(w) for w in nltk.word_tokenize(clean_text(s)) if w not in STOP])

df['text_clean'] = df[text_col].apply(clean_text)
df['text_stem'] = df['text_clean'].apply(stem_text)

print("Cleaned and processed columns ready:", df.columns.tolist())
df.head(3)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
/tmp/ipython-input-950278339.py:12: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("r_dataisbeautiful_posts.csv")


Data shape: (193091, 12)
Cleaned and processed columns ready: ['id', 'title', 'score', 'author', 'author_flair_text', 'removed_by', 'total_awards_received', 'awarders', 'created_utc', 'full_link', 'num_comments', 'over_18', 'text_len', 'word_count', 'text_clean', 'text_stem']


,id,title,score,author,author_flair_text,removed_by,total_awards_received,awarders,created_utc,full_link,num_comments,over_18,text_len,word_count,text_clean,text_stem
0,it7yz2,[OC] The World’s Forests Mapped,1,vividmaps,NaN,0,0.0,[],1600173678,https://www.reddit.com/r/dataisbeautiful/comme...,1,False,31,5,the world s forests mapped,world forest map
1,it7uig,[OC] Airbnb presence mapped in Barcelona (BCN)...,1,JonnieNeptune,OC: 2,0,0.0,[],1600173208,https://www.reddit.com/r/dataisbeautiful/comme...,0,False,108,14,airbnb presence mapped in barcelona bcn spain ...,airbnb presenc map barcelona bcn spain natur l...
2,it7t4z,A marketplace for open streaming data sources,1,DangerMouse289,NaN,1,0.0,[],1600173075,https://www.reddit.com/r/dataisbeautiful/comme...,0,False,45,7,a marketplace for open streaming data sources,marketplac open stream data sourc


# Splitting Data and Imports for ML

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, precision_recall_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

RANDOM_STATE = 42

X = df.copy()
y = df[target_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)
print("Train/Test split:", X_train.shape, X_test.shape)


Train/Test split: (154472, 16) (38619, 16)


# Building TF-IDF + Metadata Feature Union

In [8]:
from sklearn.pipeline import make_pipeline

# TF-IDF vectorizers
tfidf_word = TfidfVectorizer(
    max_features=40000,
    ngram_range=(1, 2),
    analyzer='word'
)
tfidf_char = TfidfVectorizer(
    max_features=15000,
    ngram_range=(3, 5),
    analyzer='char'
)

# Helper transformer for numeric features
from sklearn.base import BaseEstimator, TransformerMixin
class MetaSelector(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
    def fit(self, X, y=None): return self
    def transform(self, X):
        return X[self.cols].astype(float).values

meta_selector = MetaSelector(meta_cols + ['text_len', 'word_count'])

combined_features = FeatureUnion([
    ('word_tfidf', Pipeline([
        ('extract', FunctionTransformer(lambda d: d['text_stem'], validate=False)),
        ('tfidf', tfidf_word)
    ])),
    ('char_tfidf', Pipeline([
        ('extract', FunctionTransformer(lambda d: d['text_clean'], validate=False)),
        ('tfidf', tfidf_char)
    ])),
    ('meta', Pipeline([
        ('extract', meta_selector),
        ('scale', StandardScaler())
    ]))
])


# SMOTE + Random Forest Pipeline

In [9]:
smote_rf_pipeline = ImbPipeline(steps=[
    ('features', combined_features),
    ('smote', SMOTE(random_state=RANDOM_STATE)),
    ('clf', RandomForestClassifier(
        n_estimators=300,
        max_depth=25,
        class_weight='balanced',
        n_jobs=-1,
        random_state=RANDOM_STATE
    ))
])

smote_rf_pipeline.fit(X_train, y_train)
y_pred_rf = smote_rf_pipeline.predict(X_test)

print("Initial RandomForest + SMOTE F1:", f1_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Initial RandomForest + SMOTE F1: 0.4089893914014517
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     35482
           1       0.36      0.47      0.41      3137

    accuracy                           0.89     38619
   macro avg       0.66      0.70      0.67     38619
weighted avg       0.90      0.89      0.90     38619



# Threshold Tuning for Best F1

In [10]:
# Only if classifier supports predict_proba
probas = smote_rf_pipeline.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, probas)
f1s = 2 * precision * recall / (precision + recall + 1e-9)

best_idx = f1s.argmax()
best_thresh = thresholds[best_idx]
print("Best threshold:", best_thresh, "Best F1:", f1s[best_idx])

y_best = (probas >= best_thresh).astype(int)
print("\nOptimized F1:", f1_score(y_test, y_best))
print(classification_report(y_test, y_best))


Best threshold: 0.5073685439799134 Best F1: 0.4133352571173992

Optimized F1: 0.41333525761293116
              precision    recall  f1-score   support

           0       0.95      0.93      0.94     35482
           1       0.38      0.46      0.41      3137

    accuracy                           0.89     38619
   macro avg       0.66      0.69      0.68     38619
weighted avg       0.90      0.89      0.90     38619



# Feature Importance (optional visualization)

In [11]:
# Extract trained RF model and show top features
rf = smote_rf_pipeline.named_steps['clf']
print("Number of features used:", rf.n_features_in_)

# Note: TF-IDF features are many, so metadata importance matters most
if len(meta_cols) > 0:
    imp = rf.feature_importances_[-len(meta_cols):]
    for col, val in zip(meta_cols, imp):
        print(f"{col:25s} -> {val:.4f}")


Number of features used: 55006
score                     -> 0.0000
num_comments              -> 0.1221
total_awards_received     -> 0.0027
created_utc               -> 0.0033


# Quick Evaluation Summary

In [13]:
from sklearn.metrics import confusion_matrix

print("Final Evaluation Summary")
print("Threshold optimized F1:", f1_score(y_test, y_best))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_best))
print("Classification Report:\n", classification_report(y_test, y_best))


Final Evaluation Summary
Threshold optimized F1: 0.41333525761293116
Confusion Matrix:
 [[33122  2360]
 [ 1705  1432]]
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94     35482
           1       0.38      0.46      0.41      3137

    accuracy                           0.89     38619
   macro avg       0.66      0.69      0.68     38619
weighted avg       0.90      0.89      0.90     38619



# 5-Fold Cross-Validation F1 (Robust Mean ± Std)

In [14]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

# Define Stratified K-Fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
f1_scorer = make_scorer(f1_score)

# Evaluate with cross-validation
cv_scores = cross_val_score(smote_rf_pipeline, X_train, y_train, cv=cv, scoring=f1_scorer, n_jobs=-1)
print("Cross-Validation F1-scores:", cv_scores)
print("Mean F1:", cv_scores.mean(), " | Std:", cv_scores.std())


Cross-Validation F1-scores: [0.39578207 0.41242938 0.41780944 0.39605356 0.3819601 ]
Mean F1: 0.4008069108536064  | Std: 0.012862450673772131


# Tuning Random Forest to Handle Class 1 Better

In [15]:
smote_rf_pipeline.set_params(
    clf__n_estimators=200,
    clf__max_depth=30,
    clf__min_samples_leaf=2,
    clf__class_weight={0:1, 1:4}   # give more weight to 'removed' class
)

smote_rf_pipeline.fit(X_train, y_train)
y_pred_tuned = smote_rf_pipeline.predict(X_test)
print("Tuned RF + SMOTE F1:", f1_score(y_test, y_pred_tuned))
print(classification_report(y_test, y_pred_tuned))


Tuned RF + SMOTE F1: 0.2764366183849285
              precision    recall  f1-score   support

           0       0.99      0.58      0.73     35482
           1       0.16      0.93      0.28      3137

    accuracy                           0.60     38619
   macro avg       0.58      0.75      0.50     38619
weighted avg       0.92      0.60      0.69     38619



# Adding Logistic Regression Ensemble (Vote Average)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

# Define base models
log_model = LogisticRegression(max_iter=1000, solver='saga', class_weight='balanced')
rf_model = RandomForestClassifier(
    n_estimators=200, max_depth=30, class_weight={0:1, 1:4},
    random_state=RANDOM_STATE, n_jobs=-1
)

# Fit text + meta features once
feat_transformer = smote_rf_pipeline.named_steps['features']
X_train_feats = feat_transformer.fit_transform(X_train)
X_test_feats = feat_transformer.transform(X_test)

# Ensemble
voting = VotingClassifier(estimators=[
    ('lr', log_model),
    ('rf', rf_model)
], voting='soft', n_jobs=-1)

voting.fit(X_train_feats, y_train)
y_vote = voting.predict(X_test_feats)

print("Ensemble F1:", f1_score(y_test, y_vote))
print(classification_report(y_test, y_vote))


Ensemble F1: 0.5960850347295307
              precision    recall  f1-score   support

           0       0.99      0.90      0.94     35482
           1       0.44      0.90      0.60      3137

    accuracy                           0.90     38619
   macro avg       0.72      0.90      0.77     38619
weighted avg       0.95      0.90      0.92     38619

